# Vulnerability Management Overview

Vulnerability management is a continuous process of identifying, evaluating, treating, and reporting security vulnerabilities in systems and software.

## Vulnerability Management Lifecycle

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│   DISCOVER  │───▶│   ASSESS    │───▶│ PRIORITIZE  │───▶│  REMEDIATE  │───▶│   VERIFY    │
│             │    │             │    │             │    │             │    │             │
│ • Asset     │    │ • CVSS      │    │ • Risk      │    │ • Patch     │    │ • Rescan    │
│   inventory │    │   scoring   │    │   scoring   │    │ • Mitigate  │    │ • Validate  │
│ • Scanning  │    │ • Context   │    │ • Business  │    │ • Accept    │    │ • Report    │
│ • Detection │    │   analysis  │    │   impact    │    │ • Transfer  │    │ • Metrics   │
└─────────────┘    └─────────────┘    └─────────────┘    └─────────────┘    └─────────────┘
       ▲                                                                           │
       └───────────────────────────────────────────────────────────────────────────┘
                                    Continuous Loop
```

| Phase | Key Activities | Tools/Methods |
|-------|----------------|---------------|
| **Discover** | Asset inventory, vulnerability scanning | Nessus, Qualys, OpenVAS |
| **Assess** | CVSS scoring, contextual analysis | NVD, vendor advisories |
| **Prioritize** | Risk ranking, business impact | EPSS, SSVC, asset criticality |
| **Remediate** | Patching, configuration changes | WSUS, Ansible, manual fixes |
| **Verify** | Rescanning, validation | Automated rescans, pen testing |

## CVSS - Common Vulnerability Scoring System

CVSS v3.1 provides a standardized method to capture vulnerability characteristics and produce numerical scores reflecting severity.

### CVSS Metric Groups

| Metric Group | Purpose | Components |
|--------------|---------|------------|
| **Base** | Intrinsic qualities (constant) | Attack Vector, Complexity, Privileges, User Interaction, Scope, CIA Impact |
| **Temporal** | Time-dependent factors | Exploit Code Maturity, Remediation Level, Report Confidence |
| **Environmental** | Organization-specific | Modified Base + CIA Requirements |

### Base Score Metrics

```
EXPLOITABILITY METRICS              IMPACT METRICS
├── Attack Vector (AV)              ├── Confidentiality (C)
│   N=Network, A=Adjacent,          │   H=High, L=Low, N=None
│   L=Local, P=Physical             │
├── Attack Complexity (AC)          ├── Integrity (I)
│   L=Low, H=High                   │   H=High, L=Low, N=None
├── Privileges Required (PR)        │
│   N=None, L=Low, H=High           └── Availability (A)
├── User Interaction (UI)               H=High, L=Low, N=None
│   N=None, R=Required              
└── Scope (S)
    U=Unchanged, C=Changed
```

### Severity Ratings

| Score Range | Severity | Typical Response Time |
|-------------|----------|----------------------|
| 0.0 | None | No action required |
| 0.1 - 3.9 | Low | 90 days |
| 4.0 - 6.9 | Medium | 30 days |
| 7.0 - 8.9 | High | 7-14 days |
| 9.0 - 10.0 | Critical | 24-72 hours |

In [ ]:
# CVSS v3.1 Base Score Calculator

def calculate_cvss_base_score(
    attack_vector: str,      # N, A, L, P
    attack_complexity: str,  # L, H
    privileges_required: str, # N, L, H
    user_interaction: str,   # N, R
    scope: str,              # U, C
    confidentiality: str,    # H, L, N
    integrity: str,          # H, L, N
    availability: str        # H, L, N
) -> dict:
    """Calculate CVSS v3.1 Base Score."""
    
    # Exploitability metric values
    av_values = {'N': 0.85, 'A': 0.62, 'L': 0.55, 'P': 0.2}
    ac_values = {'L': 0.77, 'H': 0.44}
    ui_values = {'N': 0.85, 'R': 0.62}
    
    # Privileges Required (depends on Scope)
    pr_values_unchanged = {'N': 0.85, 'L': 0.62, 'H': 0.27}
    pr_values_changed = {'N': 0.85, 'L': 0.68, 'H': 0.50}
    
    # Impact metric values
    cia_values = {'H': 0.56, 'L': 0.22, 'N': 0}
    
    # Select PR values based on Scope
    pr_values = pr_values_changed if scope == 'C' else pr_values_unchanged
    
    # Calculate Exploitability Sub-score
    exploitability = 8.22 * av_values[attack_vector] * ac_values[attack_complexity] * \
                     pr_values[privileges_required] * ui_values[user_interaction]
    
    # Calculate Impact Sub-score
    isc_base = 1 - ((1 - cia_values[confidentiality]) * 
                    (1 - cia_values[integrity]) * 
                    (1 - cia_values[availability]))
    
    if scope == 'U':
        impact = 6.42 * isc_base
    else:  # Scope Changed
        impact = 7.52 * (isc_base - 0.029) - 3.25 * pow(isc_base - 0.02, 15)
    
    # Calculate Base Score
    if impact <= 0:
        base_score = 0
    elif scope == 'U':
        base_score = min(impact + exploitability, 10)
    else:
        base_score = min(1.08 * (impact + exploitability), 10)
    
    # Round up to 1 decimal place
    import math
    base_score = math.ceil(base_score * 10) / 10
    
    # Determine severity
    if base_score == 0:
        severity = 'None'
    elif base_score <= 3.9:
        severity = 'Low'
    elif base_score <= 6.9:
        severity = 'Medium'
    elif base_score <= 8.9:
        severity = 'High'
    else:
        severity = 'Critical'
    
    vector_string = f"CVSS:3.1/AV:{attack_vector}/AC:{attack_complexity}/" \
                    f"PR:{privileges_required}/UI:{user_interaction}/S:{scope}/" \
                    f"C:{confidentiality}/I:{integrity}/A:{availability}"
    
    return {
        'base_score': base_score,
        'severity': severity,
        'exploitability': round(exploitability, 2),
        'impact': round(impact, 2),
        'vector_string': vector_string
    }

In [ ]:
# Example CVSS Calculations for Real-World Vulnerabilities

vulnerabilities = {
    'Log4Shell (CVE-2021-44228)': {
        'attack_vector': 'N', 'attack_complexity': 'L',
        'privileges_required': 'N', 'user_interaction': 'N',
        'scope': 'C', 'confidentiality': 'H', 'integrity': 'H', 'availability': 'H'
    },
    'Heartbleed (CVE-2014-0160)': {
        'attack_vector': 'N', 'attack_complexity': 'L',
        'privileges_required': 'N', 'user_interaction': 'N',
        'scope': 'U', 'confidentiality': 'H', 'integrity': 'N', 'availability': 'N'
    },
    'Local Privilege Escalation': {
        'attack_vector': 'L', 'attack_complexity': 'L',
        'privileges_required': 'L', 'user_interaction': 'N',
        'scope': 'U', 'confidentiality': 'H', 'integrity': 'H', 'availability': 'H'
    },
    'XSS (Stored)': {
        'attack_vector': 'N', 'attack_complexity': 'L',
        'privileges_required': 'L', 'user_interaction': 'R',
        'scope': 'C', 'confidentiality': 'L', 'integrity': 'L', 'availability': 'N'
    }
}

print("CVSS Base Score Calculations\n" + "=" * 60)
for name, params in vulnerabilities.items():
    result = calculate_cvss_base_score(**params)
    print(f"\n{name}")
    print(f"  Score: {result['base_score']} ({result['severity']})")
    print(f"  Vector: {result['vector_string']}")
    print(f"  Exploitability: {result['exploitability']} | Impact: {result['impact']}")

## Vulnerability Scanning Types

### Scanning Method Comparison

| Type | Full Name | When Applied | Strengths | Limitations |
|------|-----------|--------------|-----------|-------------|
| **SAST** | Static Application Security Testing | Development (code) | Early detection, covers all code paths | High false positives, no runtime context |
| **DAST** | Dynamic Application Security Testing | Runtime (deployed) | Finds runtime issues, low false positives | Limited coverage, requires running app |
| **SCA** | Software Composition Analysis | Build/CI pipeline | Identifies vulnerable dependencies | Only finds known CVEs in libraries |
| **IAST** | Interactive AST | Runtime with agent | Combines SAST+DAST benefits | Performance overhead, complex setup |

```
Development Lifecycle & Scanning Integration

  CODE        BUILD         TEST          DEPLOY        PRODUCTION
   │           │             │              │               │
   ▼           ▼             ▼              ▼               ▼
┌──────┐   ┌──────┐     ┌──────┐      ┌──────┐        ┌──────┐
│ SAST │   │ SCA  │     │ DAST │      │ DAST │        │ Vuln │
│      │   │      │     │ IAST │      │      │        │ Scan │
└──────┘   └──────┘     └──────┘      └──────┘        └──────┘
   │           │             │              │               │
   └───────────┴─────────────┴──────────────┴───────────────┘
                    Shift Left Security
```

### Common Scanning Tools

| Category | Open Source | Commercial |
|----------|-------------|------------|
| **SAST** | Semgrep, Bandit, SonarQube | Checkmarx, Fortify, Veracode |
| **DAST** | OWASP ZAP, Nikto, Nuclei | Burp Suite, Acunetix, Netsparker |
| **SCA** | OWASP Dependency-Check, Snyk (free tier) | Black Duck, WhiteSource, Snyk |
| **Infrastructure** | OpenVAS, Nmap | Nessus, Qualys, Rapid7 |

## CVE & CWE Databases

### CVE - Common Vulnerabilities and Exposures

Unique identifiers for publicly known security vulnerabilities.

```
CVE ID Format: CVE-YYYY-NNNNN
               │    │    │
               │    │    └── Sequential number (4-7 digits)
               │    └─────── Year assigned
               └──────────── Prefix

Example: CVE-2021-44228 (Log4Shell)
```

### CWE - Common Weakness Enumeration

Categories of software/hardware weaknesses that can lead to vulnerabilities.

| CWE ID | Weakness | Description |
|--------|----------|-------------|
| CWE-79 | XSS | Cross-site Scripting |
| CWE-89 | SQLi | SQL Injection |
| CWE-22 | Path Traversal | Improper path limitation |
| CWE-287 | Improper Auth | Authentication bypass |
| CWE-502 | Deserialization | Untrusted data deserialization |
| CWE-798 | Hardcoded Creds | Embedded credentials |

### Key Vulnerability Databases

| Database | Maintainer | URL | Key Features |
|----------|------------|-----|---------------|
| **NVD** | NIST | nvd.nist.gov | CVSS scores, CPE matching |
| **CVE** | MITRE | cve.org | Authoritative CVE list |
| **OSV** | Google | osv.dev | Open source vulnerabilities |
| **GitHub Advisory** | GitHub | github.com/advisories | Package-focused advisories |
| **CISA KEV** | CISA | cisa.gov/known-exploited-vulnerabilities | Actively exploited vulns |

## Patch Management & Remediation

### Patch Management Workflow

```
┌────────────────┐     ┌────────────────┐     ┌────────────────┐
│  1. IDENTIFY   │────▶│   2. ASSESS    │────▶│   3. TEST      │
│                │     │                │     │                │
│ • Vendor       │     │ • Criticality  │     │ • Staging env  │
│   advisories   │     │ • Affected     │     │ • Regression   │
│ • CVE feeds    │     │   systems      │     │ • Compatibility│
└────────────────┘     └────────────────┘     └────────────────┘
                                                      │
        ┌─────────────────────────────────────────────┘
        ▼
┌────────────────┐     ┌────────────────┐     ┌────────────────┐
│   4. DEPLOY    │────▶│  5. VERIFY     │────▶│  6. DOCUMENT   │
│                │     │                │     │                │
│ • Maintenance  │     │ • Confirm      │     │ • Update CMDB  │
│   windows      │     │   installation │     │ • Audit trail  │
│ • Rollback     │     │ • Rescan       │     │ • Compliance   │
│   plan         │     │ • Functional   │     │   reporting    │
└────────────────┘     └────────────────┘     └────────────────┘
```

### Remediation Options

| Strategy | When to Use | Example |
|----------|-------------|----------|
| **Patch** | Fix available, tested | Apply security update |
| **Mitigate** | Patch unavailable/risky | WAF rule, disable feature |
| **Accept** | Low risk, high remediation cost | Document and monitor |
| **Transfer** | Cannot remediate internally | Cyber insurance, outsource |

### Prioritization Frameworks

| Framework | Developer | Focus |
|-----------|-----------|-------|
| **CVSS** | FIRST | Technical severity |
| **EPSS** | FIRST | Exploitation probability (30-day window) |
| **SSVC** | CISA/CMU | Stakeholder-specific decision trees |
| **CISA KEV** | CISA | Known exploited vulnerabilities |

In [ ]:
# Vulnerability Prioritization Model

def calculate_risk_priority(
    cvss_score: float,
    epss_score: float,  # 0-1 probability
    asset_criticality: str,  # critical, high, medium, low
    is_internet_facing: bool,
    in_kev_catalog: bool
) -> dict:
    """Calculate risk-based vulnerability priority."""
    
    # Asset criticality multipliers
    criticality_weights = {'critical': 1.5, 'high': 1.2, 'medium': 1.0, 'low': 0.7}
    
    # Base risk = CVSS * EPSS weight
    epss_multiplier = 1 + (epss_score * 2)  # 1.0 to 3.0
    base_risk = cvss_score * epss_multiplier
    
    # Context adjustments
    context_multiplier = criticality_weights[asset_criticality]
    if is_internet_facing:
        context_multiplier *= 1.3
    if in_kev_catalog:
        context_multiplier *= 2.0  # Known exploited = urgent
    
    risk_score = min(base_risk * context_multiplier, 100)
    
    # Determine priority and SLA
    if risk_score >= 70 or in_kev_catalog:
        priority, sla = 'P1 - Critical', '24-72 hours'
    elif risk_score >= 50:
        priority, sla = 'P2 - High', '7 days'
    elif risk_score >= 30:
        priority, sla = 'P3 - Medium', '30 days'
    else:
        priority, sla = 'P4 - Low', '90 days'
    
    return {
        'risk_score': round(risk_score, 1),
        'priority': priority,
        'remediation_sla': sla
    }

# Example prioritization
test_vulns = [
    {'name': 'Log4Shell on prod web server', 'cvss': 10.0, 'epss': 0.975, 
     'asset': 'critical', 'internet': True, 'kev': True},
    {'name': 'Medium SQLi on internal app', 'cvss': 6.5, 'epss': 0.15, 
     'asset': 'medium', 'internet': False, 'kev': False},
    {'name': 'High RCE on dev server', 'cvss': 8.1, 'epss': 0.02, 
     'asset': 'low', 'internet': False, 'kev': False},
]

print("Risk-Based Prioritization Results\n" + "=" * 60)
for v in test_vulns:
    result = calculate_risk_priority(v['cvss'], v['epss'], v['asset'], v['internet'], v['kev'])
    print(f"\n{v['name']}")
    print(f"  CVSS: {v['cvss']} | EPSS: {v['epss']:.1%}")
    print(f"  Risk Score: {result['risk_score']} → {result['priority']}")
    print(f"  Remediation SLA: {result['remediation_sla']}")

## Security Metrics & KPIs

### Key Vulnerability Management Metrics

| Metric | Formula | Target | Purpose |
|--------|---------|--------|----------|
| **MTTR** | Σ(Remediation Time) / Count | < 7 days (critical) | Speed of response |
| **Vulnerability Density** | Vulns / Assets | Decreasing trend | Overall hygiene |
| **Patch Coverage** | Patched Assets / Total | > 95% | Patch effectiveness |
| **SLA Compliance** | On-time / Total | > 90% | Process adherence |
| **Recurrence Rate** | Reintroduced / Fixed | < 5% | Prevention quality |
| **Scan Coverage** | Scanned / Total Assets | 100% | Visibility |

### Sample Dashboard Metrics

```
┌─────────────────────────────────────────────────────────────────┐
│              VULNERABILITY MANAGEMENT DASHBOARD                  │
├────────────────┬────────────────┬────────────────┬──────────────┤
│   CRITICAL     │     HIGH       │    MEDIUM      │     LOW      │
│      12        │      47        │     156        │     289      │
│   ▼ 3 (20%)    │   ▲ 5 (12%)    │   ▼ 12 (7%)    │   ─ (0%)     │
├────────────────┴────────────────┴────────────────┴──────────────┤
│  MTTR (Critical): 2.3 days  │  SLA Compliance: 94%             │
│  Scan Coverage: 98%         │  Patch Rate (30d): 87%           │
└─────────────────────────────────────────────────────────────────┘
```

### Reporting Cadence

| Report Type | Audience | Frequency | Key Content |
|-------------|----------|-----------|-------------|
| Executive Summary | CISO, Board | Monthly | Risk trends, top risks, compliance |
| Technical Report | Security Team | Weekly | New vulns, remediation status |
| Operational Dashboard | SOC, IT Ops | Daily | Open items, SLA breaches |
| Compliance Report | Auditors | Quarterly | Policy adherence, evidence |

## Key Takeaways

1. **Lifecycle Approach**: Vulnerability management is continuous—discover, assess, prioritize, remediate, verify, repeat

2. **CVSS Limitations**: Base scores alone are insufficient; use EPSS, asset context, and KEV catalog for prioritization

3. **Shift Left**: Integrate SAST/SCA early in development; DAST for runtime validation

4. **Risk-Based Prioritization**: Focus on exploitability (EPSS) + asset criticality, not just severity

5. **Measure What Matters**: Track MTTR, SLA compliance, and vulnerability density trends

### Quick Reference - CVSS v3.1 Vector String

```
CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:C/C:H/I:H/A:H = 10.0 Critical
        │    │    │    │    │   │   │   │
        │    │    │    │    │   │   │   └─ Availability: High
        │    │    │    │    │   │   └───── Integrity: High  
        │    │    │    │    │   └───────── Confidentiality: High
        │    │    │    │    └───────────── Scope: Changed
        │    │    │    └────────────────── User Interaction: None
        │    │    └─────────────────────── Privileges Required: None
        │    └──────────────────────────── Attack Complexity: Low
        └───────────────────────────────── Attack Vector: Network
```